# TP3 - Prédiction de prix en finance de marché

Dans ce notebook, nous allons implémenter plusieurs variantes de réseaux de neurones récurrents pour prédire le prix d'une action. 


## Préparation

Commençons par charger les données et les visualiser.

In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd

def get_price(tickers, period="5y"):
    prices = yf.download(tickers, period=period, auto_adjust=True, progress=False)["Close"]
    if prices.empty:
        raise ValueError("No data returned")
    prices = prices.dropna()
    prices.index.name = "date"

    return prices


tickers = ["TSM", "NVDA", "META"]
df = get_price(tickers=tickers, period="5y")
df.head()

Visualisons les différents séries :

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style(style="whitegrid")



plt.figure(figsize=(15, 5))

for index, ticker in enumerate(tickers):
    y = df[ticker]

    plt.subplot(1, len(tickers), index+1)
    plt.plot(y, c=sns.color_palette()[0])
    plt.title(ticker)
    plt.ylabel("Price")

plt.show()

## Baselines

Avant de s'attaquer aux réseaux de neurones, on se propose deux baselines *naïves* :
- Prédire le prix de la veille
- Faire une moyenne glissante sur 7 jours

Nous mesurerons la performance à l'aide de la RMSE.

In [ ]:
from sklearn.metrics import root_mean_squared_error as RMSE

train_size = 0.8



plt.figure(figsize=(15, 5))

for index, ticker in enumerate(tickers):
    y = df[ticker]
    split = int(train_size * len(y))
    y_train, y_test = y.iloc[:split], y.iloc[split:]

    y_pred_lag1 = y_test.shift(1)
    y_pred_lag1.iloc[0] = y_train.iloc[-1]

    history = pd.concat([y_train, y_test])
    y_pred_roll7 = history.shift(1).rolling(7).mean().iloc[len(y_train):]

    rmse_lag = RMSE(y_test, y_pred_lag1)
    rmse_roll = RMSE(y_test, y_pred_roll7)



    plt.subplot(1, len(tickers), index+1)
    plt.plot(y_test, alpha=0.8, label="True", lw=2)
    plt.plot(y_pred_lag1, alpha=0.8, label=f"Lag-1 (RMSE={rmse_lag:.2f})")
    plt.plot(y_pred_roll7, alpha=0.8, label=f"Roll7 (RMSE={rmse_roll:.2f})")
    plt.title(f"{ticker} (mean: {np.mean(y_test):.2f})")
    plt.ylabel("Price")
    plt.legend()

plt.show()


Nous avons déjà des performances très acceptables ! Voyons si un réseau de neurones peut faire mieux.

## Réseau de neurones récurrents *simple*

On commence par un réseau de neurones récurrents simple, nous traiterons GRU, LSTM et convolutionnel plus tard. Pour le moment, nous allons prédire le prix d'une action en utilisant uniquement les précédents prix de l'action.

### Préparation

Pour obtenir un dataset d'entraînement et de tests, nous devons :
1. Découper le dataset actuel en respectant la chronologie des données
2. Normaliser les données : à la fois les features et la cible !

Puisque nous souhaitons visualiser les performances de prédiction, nous devons garder en tête que les prédictions devrons être *re-scalées* avec la méthode [`inverse_transform`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.inverse_transform) de la classe [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

In [ ]:
from sklearn.preprocessing import StandardScaler

def get_dataset(df, ticker, window_size, train_ratio=0.8):
    y = df[ticker].values.reshape(-1, 1)

    X, Y = [], []
    for i in range(window_size, len(y)):
        X.append(y[i-window_size:i])
        Y.append(y[i])

    X = np.array(X)
    Y = np.array(Y)

    split = int(train_ratio * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = Y[:split], Y[split:]

    X_scaler = StandardScaler()
    y_scaler = StandardScaler()

    X_train = X_scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
    X_test = X_scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

    y_train = y_scaler.fit_transform(y_train)
    y_test = y_scaler.transform(y_test)

    return X_train, X_test, y_train, y_test, y_scaler


### Modélisation

On se propose de définir un modèle avec 64 neurones dans une couche [`SimpleRNN`](https://keras.io/api/layers/recurrent_layers/simple_rnn/).

**Consigne** : Définir le modèle souhaité, puis entraîner le modèle sur l'action NVDA (Nvidia) avec une fenêtre glissante de taille 10 (on prendra 30 époques et un batch size de 32). Puis sauvegarder les prédictions du modèle dans un vector `y_pred`.

On souhaite visualiser la qualité de la prédiction ainsi que la performance métrique du modèle.

**Consigne** : Réaliser un graphique comparant le véritable prix de l'action et la prédiction du modèle, en affichant la RMSE du modèle ainsi que la moyenne des prix de l'action.

Il semblerait que le modèle soit *en retard* par rapport aux prix. Cependant les tendances semblent correctes au début, mais de plus en plus éloignées au fur et à mesure du temps.
Il est possible que ça soit dû à un entraînement trop court du modèle. Vérifions-le.

**Consigne** : Produire un graphique similaire en affichant quatres courbes :
1. Le prix de l'action
2. La prédiction du modèle après 10 époques d'entraînement
3. La prédiction du modèle après 30 époques d'entraînement
4. La prédiction du modèle après 50 époques d'entraînement

Pour éviter d'entraîner plusieurs modèles différents, on peut réentraîner un modèle pour un nombre d'époques spécifique.

In [ ]:
ticker = "NVDA"
window_size = 10

X_train, X_test, y_train, y_test, y_scaler = get_dataset(df, ticker, window_size)

model = Sequential([
        layers.Input(shape=(window_size, 1)),
        layers.SimpleRNN(units=64),
        layers.Dense(1)
    ])
model.compile(optimizer=Adam(1e-3), loss="mse")

history = model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
y_pred_10 = model.predict(X_test)

history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
y_pred_30 = model.predict(X_test)

history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
y_pred_50 = model.predict(X_test)


plt.figure(figsize=(12, 5))
plt.plot(y_scaler.inverse_transform(y_test), label="True", lw=2)
plt.plot(y_scaler.inverse_transform(y_pred_10), label=f"10 epochs (RMSE: {RMSE(y_scaler.inverse_transform(y_test), y_scaler.inverse_transform(y_pred_10)):.2f})", alpha=0.6)
plt.plot(y_scaler.inverse_transform(y_pred_30), label=f"30 epochs (RMSE: {RMSE(y_scaler.inverse_transform(y_test), y_scaler.inverse_transform(y_pred_30)):.2f})", alpha=0.6)
plt.plot(y_scaler.inverse_transform(y_pred_50), label=f"50 epochs (RMSE: {RMSE(y_scaler.inverse_transform(y_test), y_scaler.inverse_transform(y_pred_50)):.2f})", alpha=0.6)
plt.ylabel("Price")
plt.title(f"{ticker} (mean: {np.mean(y_scaler.inverse_transform(y_test)):.2f})")
plt.legend()
plt.show()

Effectivement, un entraînement plus long permet d'être plus précis sur les dates les plus lointaines de la fin de l'entraînement.

Maintenant que nous avons réussi à prédire le prix d'une action avec un type de couche récurrente, attaquons-nous aux autres actions avec l'ensemble des types de couches.

## Comparaisons des différentes couches

Nous allons utiliser les couches [`SimpleRNN`](https://keras.io/api/layers/recurrent_layers/simple_rnn/), [`GRU`](https://keras.io/api/layers/recurrent_layers/gru/), [`LSTM`](https://keras.io/api/layers/recurrent_layers/lstm/) et [`Conv1D`](https://keras.io/api/layers/convolution_layers/conv1d/).
A part la couche de convolution, les autres couches peuvent être interchangées dans la définition d'un modèle. Exploitant cela, nous proposons la fonction suivante pour obtenir un modèle compilé :

In [ ]:
def get_model(layer, window_size, learning_rate=1e-3):
    if layer == layers.Conv1D:
        model = Sequential([
            layers.Input(shape=(window_size, 1)),
            layers.Conv1D(64, kernel_size=3, activation="relu"),
            layers.Flatten(),
            layers.Dense(1)
        ])
    else:
        model = Sequential([
            layers.Input(shape=(window_size, 1)),
            layer(64),
            layers.Dense(1)
        ])
    model.compile(optimizer=Adam(learning_rate), loss="mse")
    return model

**Consigne** : A partir des éléments décrit précédemment, produire un graphique où pour les trois actions sont affichés les performances de chaque type de couche.

## Predict with other features

Jusqu'ici nous n'avons utilisé que les prix précédents, et si nous utilisions les prix des autres actions aussi ?

**Consigne** : Adapter la fonction `get_dataset` en ajoutant un paramètre *features* qui correspond à une liste de chaîne de caractère représentant les noms des actions (autre que celle d'intérêt).

**Consigne** : Adapter la fonction `get_model` pour prendre en compte le nombre de features total.

**Consigne** : A partir des éléments décrit précédemment, produire un graphique où pour les trois actions sont affichés les performances de chaque type de couche. Cette fois en utilisant l'ensemble des informations disponibles.